In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup

# Importando classe Amazon criada
from amazon import Amazon

# Teclas
from selenium.webdriver.common.keys import Keys

# Pandas
import pandas as pd

# AWS .ENV
import os
from dotenv import load_dotenv

In [52]:
# Iniciando o .env
load_dotenv()

True

Iniciando o Scraping

In [53]:
url = 'https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/#onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,&ordenar-por=preco:ASC&preco-ate=150000'
options = Options()
options.add_argument('window-size=1200,800')
driver = webdriver.Chrome(options=options)
houses = []

def iniciaScrap(url, houseValue) -> None:
    driver.get(url)
    filterValue(houseValue)
    getAllCards()
    driver.close()

# Coloca o valor que eu quero no input
def filterValue(value) -> None:
    input_element = driver.find_element(By.CSS_SELECTOR, '#filter-range-to-price')
    input_element.send_keys(value)
    input_element.send_keys(Keys.ENTER)

# Pegando o valor dos cards
def getAllCards():
    while True:
        if(len(houses) >= 500): 
            break
        sleep(2)
        page = BeautifulSoup(driver.page_source, 'html.parser')
        cards = page.findAll('div', attrs={'data-type': 'property'})
        for card in cards:
            card_title = card.find('span', class_='property-card__title js-cardLink js-card-title')
            m2, qtd_quarto, qtd_banheiro, vaga_carro = card.find('ul', class_='property-card__details').findAll('span', class_='property-card__detail-value')
            valor = card.find('section', class_='property-card__values').find('div', class_='property-card__price js-property-card-prices js-property-card__price-small').find('p')
            houses.append({'titulo': card_title.text.strip(), 'metro_quadrato': m2.text.strip(), 'qtd_quarto': qtd_quarto.text.strip(), 'qtd_banheiro': qtd_banheiro.text.strip(), 'qtd_vaga_carro': vaga_carro.text.strip(), 'valor': valor.contents[0].strip()})
        # desce o scroll até o fim da página 
        next_page = driver.find_element(By.CSS_SELECTOR, '#js-site-main > div.results__container > div.results__content > section > div.results-main__panel.js-list > div.js-results-pagination > div > ul > li:nth-child(9) > button')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        sleep(1)
        next_page.click()


        


The chromedriver version (125.0.6422.78) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (126.0.6478.126); currently, chromedriver 126.0.6478.126 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


In [54]:
apartament_value = input('Selecione o valor do apartamento')
iniciaScrap(url, apartament_value)
houses

[{'titulo': 'Apartamento com 2 Quartos à Venda, 48m²',
  'metro_quadrato': '48',
  'qtd_quarto': '2',
  'qtd_banheiro': '1',
  'qtd_vaga_carro': '1',
  'valor': 'R$ 50.000'},
 {'titulo': 'Apartamento com  Quarto à Venda, 60m²',
  'metro_quadrato': '60',
  'qtd_quarto': '1',
  'qtd_banheiro': '1',
  'qtd_vaga_carro': '--',
  'valor': 'R$ 50.000'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 44m²',
  'metro_quadrato': '44',
  'qtd_quarto': '2',
  'qtd_banheiro': '1',
  'qtd_vaga_carro': '--',
  'valor': 'R$ 50.000'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 48m²',
  'metro_quadrato': '48',
  'qtd_quarto': '2',
  'qtd_banheiro': '1',
  'qtd_vaga_carro': '1',
  'valor': 'R$ 55.000'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 46m²',
  'metro_quadrato': '46',
  'qtd_quarto': '2',
  'qtd_banheiro': '1',
  'qtd_vaga_carro': '1',
  'valor': 'R$ 60.000'},
 {'titulo': 'Apartamento com 2 Quartos à Venda, 47m²',
  'metro_quadrato': '47',
  'qtd_quarto': '2',
  'qtd_banheiro': '1',


Gerando arquivo .csv

In [55]:
#gera arquivo .csv na pasta dataset
df = pd.DataFrame(houses)
df.to_csv('dataset/apartamentos.csv', sep=';', index=False)

Amazon

In [58]:
# Inicia instância na amazon
am = Amazon(os.getenv('AWS_KEY_ID'), os.getenv('AWS_SECRET_KEY'))

# gera cliente no s3
s3_cliente = am.getS3Client()

In [62]:
# Fazendo upload do arquivo gerado no amazon s3
am.uploadFile(os.getenv('AWS_BUCKET_NAME'), 'dataset/apartamentos.csv', 'bronze/ingest/batch-apartaments/apartamentos.csv')